In [1]:
import numpy as np
import cv2
import tensorflow as tf
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from numpy.random import randn
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import RepeatVector
from matplotlib import pyplot
from tensorflow.keras.utils import plot_model

import time as t
from statistics import mean

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 4GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10240)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
# define the discriminator model
def define_discriminator(image_shape, vector_shape):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # source image input
    in_src_image = Input(shape=image_shape)
    # target image input
    in_target = Input(shape=(vector_shape[0], vector_shape[1]))
    # Vector layer
    n_nodes = (image_shape[0] * image_shape[1] * image_shape[2])
    # LSTM Model
    p = LSTM(200, activation='tanh')(in_target)
    #p = LSTM(200, activation=None)(in_target)
    p = Dense(n_nodes)(p)
    # Reshape and mergo to image dimension
    p = Reshape((image_shape[0], image_shape[1], image_shape[2]))(p)
    merged = Concatenate()([p, in_src_image])
    # concatenate images channel-wise
    # C64
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    # C128
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C256
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C512
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # second last output layer
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # patch output
    d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Flatten()(d)
    d = Dropout(0.4)(d)
    out_layer = Dense(1, activation='sigmoid')(d)
    # define model
    model = Model([in_src_image, in_target], out_layer)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model


In [4]:
# define an encoder block
def define_encoder_block(layer_in, n_filters, batchnorm=True):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # add downsampling layer
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same',
    kernel_initializer=init)(layer_in)
    # conditionally add batch normalization
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    # leaky relu activation
    g = LeakyReLU(alpha=0.2)(g)
    return g

In [5]:
# define the standalone generator model
def define_generator(in_shape, vector_shape, latent_dim):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=in_shape)
    in_lat = Input(shape=latent_dim)
    gen = LeakyReLU(alpha=0.2)(in_lat)
    # merge image gen and label input
    # encoder model
    e1 = define_encoder_block(in_image, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    e3 = define_encoder_block(e2, 256)
    e4 = define_encoder_block(e3, 512)
    e5 = define_encoder_block(e4, 512)
    e6 = define_encoder_block(e5, 512)
    e7 = define_encoder_block(e6, 512)
    # bottleneck, no batch norm and relu
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
    b = Activation('relu')(b)
    b = Flatten()(b)
    # Merge latent space variable input with network
    b = Concatenate()([b, gen])
    # Reshape to vector size
    n_nodes = (vector_shape[0] * vector_shape[1])
    b = Dense(n_nodes)(b)
    b = Reshape([vector_shape[0], vector_shape[1]])(b)
    # encoder-decoder LSTM model
    d = LSTM(200, activation='relu')(b)
    d = RepeatVector(vector_shape[0])(d)
    d = LSTM(200, activation='relu', return_sequences=True)(d)
    # output
    out_layer = Dense(3)(d)

    # define model input & output
    model = Model([in_image, in_lat], out_layer)
    return model

In [6]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model, image_shape, vector_shape, latent_dim):
    # make weights in the discriminator not trainable
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
    # define the source image
    in_src = Input(shape=image_shape)
    in_lat = Input(shape=latent_dim)
    # connect the source image to the generator input
    gen_out = g_model([in_src, in_lat])
    # connect the source input and generator output to the discriminator input
    dis_out = d_model([in_src, gen_out])
    # src image as input, generated image and classification output
    model = Model([in_src, in_lat], [dis_out, gen_out])
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    return model

In [7]:
# load and prepare training images
def load_real_samples(filename):
	# load compressed arrays
	data = load(filename)
	# unpack arrays
	X1, X2 = data['arr_0'], data['arr_1']
	return [X1, X2]

In [8]:
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples):
	# unpack dataset
	trainA, trainB = dataset
	# choose random instances
	ix = randint(0, trainA.shape[0], n_samples)
	# retrieve selected images
	X1, X2 = trainA[ix], trainB[ix]
	# generate 'real' class labels 
	y = ones((n_samples, 1))
	return [X1, X2], y

In [9]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [10]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, n_samples):
    # Generate latent space points
    z_input = generate_latent_points(latent_dim, n_samples)
    # generate fake instance
    X = g_model.predict([samples, z_input])
    # create 'fake' class labels 
    y = zeros((n_samples, 1))
    return X, y

In [11]:
# Overlay trajectories (data_y) to the image (data_x)
def create_trajectory(data_x, data_y, obs_len=10):
    # Calibration parameter to overlay for a 1280x360 resolution image
    K = np.array([[537.023764, 0, 640 , 0], 
                    [0 , 537.023764, 180, 0], 
                    [0, 0, 1, 0]])
    # Rotation matrix to obtain egocentric trajectory
    Rt = np.array([[0.028841, 0.007189, 0.999558, 1.481009],
                    [-0.999575,  0.004514,  0.028809,  0.296583],
                    [ 0.004305,  0.999964, -0.007316, -1.544537],
                    [ 0.      ,  0.      ,  0.      ,  1.      ]])

    # Resize data back to 1280x360
    data_x = cv2.resize(data_x, (1280,360))
    # Add column of ones for rotation matrix multiplication
    data_y = np.hstack((data_y, np.ones((len(data_y),1))))
    # Draw points
    for m in range(obs_len, data_y.shape[0]):
        # Rotation matrix multiplication of trajectory 
        A = np.matmul(np.linalg.inv(Rt), data_y[m, :].reshape(4, 1))
        # Egocentric view of trajectory
        B = np.matmul(K, A)
        # Circle location of trajectories 
        x = int(B[0, 0] * 1.0 / B[2, 0])
        y = int(B[1, 0] * 1.0 / B[2, 0])
        if (x < 0 or x > 1280 - 1 or y > 360 - 1):
            continue
        # Use opencv to overlay trajectories
        data_x = cv2.circle(data_x, (x, y), 3, (0, 0, 255), -1)
    return data_x

In [12]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, dataset, n_samples=1):
    # select a sample of input images
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples)
    # generate a batch of fake samples
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, n_samples)
    # scale all pixels from [-1,1] to [0,1]
    X_realA = (X_realA + 1) / 2.0
    # plot real source images
    for i in range(n_samples):
        orig_image = (X_realA[i]* 255).astype(np.uint8)
        orig_image = cv2.resize(orig_image, (1280,360))
        pyplot.subplot(3, n_samples, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(orig_image)
    # plot generated target image
    for i in range(n_samples):
        fake_sample = create_trajectory((X_realA[i]* 255).astype(np.uint8), X_fakeB[i])
        pyplot.subplot(3, n_samples, 1 + n_samples + i)
        pyplot.axis('off')
        pyplot.imshow(fake_sample)
    # plot real target image
    for i in range(n_samples):
        true_sample = create_trajectory((X_realA[i]* 255).astype(np.uint8), X_realB[i])
        pyplot.subplot(3, n_samples, 1 + n_samples*2 + i)
        pyplot.axis('off')
        pyplot.imshow(true_sample)
    # save plot to file
    filename1 = 'plot_%06d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    # save the generator model
    filename2 = 'model_%06d.h5' % (step+1)
    g_model.save(filename2)
    print('>Saved: %s and %s' % (filename1, filename2))

In [13]:
# train models
def train(d_model, g_model, gan_model, dataset, latent_dim, n_epochs=50, n_batch=1):
    # calculate the number of batches per training epoch
    bat_per_epo = int(len(dataset[0]) / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    #for timining
    #n_steps = 25
    times = []
    # manually enumerate epochs
    for i in range(n_steps):
        #start timer
        start = t.time()
        # select a batch of real samples
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch)
        # generate a batch of fake samples
        X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_batch)
        # update discriminator for real samples
        d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
        # update discriminator for generated samples
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
        X_lat = generate_latent_points(latent_dim, n_batch)
        # update the generator
        g_loss, _, _ = gan_model.train_on_batch([X_realA, X_lat], [y_real, X_realB])
        #end timer
        end = t.time()
        #print(f'time for itr: {end-start} sec')
        times.append(end-start)
        # summarize performance
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
        # summarize model performance
        if (i+1) % int(bat_per_epo) == 0:
            summarize_performance(i, g_model, dataset)
            print('saved')
    print(f'average time per itr: {mean(times)}')

In [14]:
%%time
# load image data
#dataset = load_real_samples('0000.npz')
dataset = load_real_samples('dataset.npz')
print('Loaded', dataset[0].shape, dataset[1].shape)

Loaded (19031, 256, 256, 3) (19031, 40, 3)
CPU times: user 25 s, sys: 690 ms, total: 25.7 s
Wall time: 25.7 s


In [15]:
# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]
vector_shape = dataset[1].shape[1:]
latent_dim = 512
print(image_shape)
print(vector_shape)

(256, 256, 3)
(40, 3)


In [16]:
# define the models
d_model = define_discriminator(image_shape, vector_shape)
g_model = define_generator(image_shape, vector_shape, latent_dim)

In [17]:
# define the composite model
gan_model = define_gan(g_model, d_model, image_shape, vector_shape, latent_dim)

In [18]:
train(d_model, g_model, gan_model, dataset, latent_dim)

>1, d1[0.148] d2[2.945] g[250.344]
>2, d1[0.447] d2[0.416] g[179.334]
>3, d1[0.122] d2[0.432] g[179.211]
>4, d1[0.156] d2[0.645] g[80.186]
>5, d1[1.354] d2[1.059] g[12.032]
>6, d1[0.861] d2[0.249] g[180.628]
>7, d1[0.089] d2[0.443] g[395.243]
>8, d1[0.602] d2[0.640] g[293.034]
>9, d1[0.000] d2[0.029] g[457.523]
>10, d1[1.260] d2[0.308] g[707.136]
>11, d1[0.119] d2[0.315] g[131.641]
>12, d1[0.145] d2[0.582] g[148.220]
>13, d1[0.363] d2[0.266] g[273.671]
>14, d1[0.086] d2[0.477] g[154.615]
>15, d1[0.223] d2[0.299] g[330.702]
>16, d1[0.024] d2[0.013] g[671.546]
>17, d1[0.010] d2[0.084] g[607.441]
>18, d1[0.828] d2[0.644] g[74.248]
>19, d1[0.000] d2[0.467] g[449.206]
>20, d1[0.003] d2[0.071] g[477.177]
>21, d1[0.323] d2[0.169] g[375.809]
>22, d1[0.080] d2[0.968] g[503.951]
>23, d1[0.116] d2[0.029] g[525.872]
>24, d1[0.099] d2[1.126] g[74.014]
>25, d1[0.192] d2[0.229] g[460.242]
>26, d1[0.207] d2[0.105] g[353.032]
>27, d1[1.673] d2[0.085] g[149.884]
>28, d1[0.057] d2[0.139] g[342.979]
>29, 

>227, d1[0.375] d2[0.448] g[133.670]
>228, d1[0.224] d2[0.182] g[322.505]
>229, d1[0.373] d2[0.172] g[248.689]
>230, d1[0.312] d2[0.153] g[81.659]
>231, d1[0.187] d2[0.851] g[69.689]
>232, d1[0.122] d2[0.174] g[225.622]
>233, d1[0.775] d2[0.475] g[389.769]
>234, d1[0.207] d2[0.247] g[217.773]
>235, d1[0.360] d2[0.169] g[459.151]
>236, d1[0.329] d2[0.173] g[257.572]
>237, d1[0.044] d2[0.189] g[313.118]
>238, d1[0.264] d2[0.001] g[259.743]
>239, d1[0.114] d2[0.278] g[72.356]
>240, d1[0.189] d2[0.003] g[325.799]
>241, d1[0.139] d2[0.002] g[332.271]
>242, d1[0.081] d2[0.009] g[139.131]
>243, d1[0.072] d2[0.001] g[135.552]
>244, d1[2.824] d2[0.008] g[295.320]
>245, d1[2.670] d2[0.032] g[277.230]
>246, d1[0.731] d2[0.149] g[166.478]
>247, d1[0.120] d2[0.043] g[490.646]
>248, d1[0.067] d2[0.099] g[460.090]
>249, d1[0.043] d2[0.121] g[194.631]
>250, d1[0.789] d2[0.039] g[114.914]
>251, d1[0.630] d2[0.125] g[205.546]
>252, d1[0.517] d2[0.778] g[23.860]
>253, d1[0.912] d2[0.123] g[139.147]
>254,

>451, d1[0.316] d2[0.276] g[166.785]
>452, d1[0.142] d2[0.394] g[393.158]
>453, d1[0.862] d2[0.132] g[105.555]
>454, d1[0.311] d2[0.112] g[195.914]
>455, d1[0.168] d2[0.265] g[71.365]
>456, d1[0.636] d2[0.480] g[112.337]
>457, d1[0.276] d2[0.494] g[254.816]
>458, d1[0.516] d2[0.594] g[82.969]
>459, d1[0.175] d2[0.912] g[127.633]
>460, d1[0.227] d2[0.449] g[105.082]
>461, d1[0.549] d2[0.237] g[289.847]
>462, d1[0.103] d2[0.406] g[164.223]
>463, d1[0.086] d2[0.130] g[91.032]
>464, d1[0.246] d2[0.407] g[472.640]
>465, d1[0.294] d2[0.065] g[303.175]
>466, d1[0.667] d2[0.088] g[203.277]
>467, d1[0.571] d2[0.676] g[146.198]
>468, d1[0.765] d2[0.532] g[81.123]
>469, d1[0.908] d2[0.832] g[226.054]
>470, d1[0.286] d2[0.523] g[99.767]
>471, d1[0.095] d2[0.164] g[271.562]
>472, d1[0.285] d2[0.195] g[126.088]
>473, d1[0.470] d2[0.482] g[117.435]
>474, d1[0.075] d2[0.117] g[123.968]
>475, d1[0.368] d2[0.383] g[293.821]
>476, d1[0.157] d2[0.440] g[501.064]
>477, d1[0.596] d2[0.586] g[285.921]
>478, 

>675, d1[0.078] d2[0.332] g[327.128]
>676, d1[0.205] d2[0.161] g[170.480]
>677, d1[0.475] d2[0.609] g[181.917]
>678, d1[0.234] d2[0.136] g[410.050]
>679, d1[0.764] d2[0.437] g[68.358]
>680, d1[0.098] d2[0.163] g[164.506]
>681, d1[0.197] d2[0.284] g[157.062]
>682, d1[0.698] d2[0.240] g[75.900]
>683, d1[0.010] d2[0.242] g[756.338]
>684, d1[0.014] d2[0.319] g[340.015]
>685, d1[0.732] d2[0.192] g[142.734]
>686, d1[0.249] d2[0.420] g[304.459]
>687, d1[0.350] d2[0.404] g[229.011]
>688, d1[0.072] d2[0.221] g[141.222]
>689, d1[0.411] d2[0.106] g[114.652]
>690, d1[0.605] d2[0.098] g[214.459]
>691, d1[0.411] d2[0.175] g[269.276]
>692, d1[0.088] d2[0.175] g[378.328]
>693, d1[0.656] d2[0.185] g[39.036]
>694, d1[0.036] d2[0.224] g[412.559]
>695, d1[0.722] d2[0.165] g[86.150]
>696, d1[0.598] d2[0.298] g[106.821]
>697, d1[0.243] d2[0.248] g[138.829]
>698, d1[0.091] d2[0.464] g[280.983]
>699, d1[0.607] d2[0.394] g[136.188]
>700, d1[0.025] d2[0.386] g[202.176]
>701, d1[0.199] d2[0.281] g[202.004]
>702,

>898, d1[0.221] d2[0.292] g[139.048]
>899, d1[0.554] d2[0.224] g[127.188]
>900, d1[0.025] d2[0.248] g[395.497]
>901, d1[0.700] d2[0.121] g[103.803]
>902, d1[0.023] d2[0.150] g[222.999]
>903, d1[0.074] d2[1.186] g[15.296]
>904, d1[0.047] d2[0.087] g[199.128]
>905, d1[0.038] d2[0.202] g[371.765]
>906, d1[0.955] d2[0.202] g[84.723]
>907, d1[0.061] d2[0.671] g[13.988]
>908, d1[0.158] d2[1.190] g[110.454]
>909, d1[0.454] d2[0.172] g[167.106]
>910, d1[0.055] d2[0.187] g[429.480]
>911, d1[0.379] d2[0.150] g[54.002]
>912, d1[0.405] d2[0.205] g[68.057]
>913, d1[0.284] d2[0.165] g[97.921]
>914, d1[0.881] d2[0.212] g[177.493]
>915, d1[0.066] d2[0.169] g[268.030]
>916, d1[0.033] d2[0.156] g[387.668]
>917, d1[0.529] d2[0.408] g[95.889]
>918, d1[0.170] d2[0.270] g[182.420]
>919, d1[0.145] d2[0.123] g[265.032]
>920, d1[0.124] d2[0.142] g[174.843]
>921, d1[0.443] d2[0.149] g[111.354]
>922, d1[0.620] d2[0.218] g[80.335]
>923, d1[0.147] d2[0.492] g[160.263]
>924, d1[0.186] d2[0.205] g[202.092]
>925, d1[

...

>16958, d1[0.545] d2[0.213] g[29.118]
>16959, d1[0.507] d2[0.361] g[22.155]
>16960, d1[0.134] d2[0.190] g[181.055]
>16961, d1[0.653] d2[0.233] g[319.948]
>16962, d1[0.112] d2[0.164] g[49.937]
>16963, d1[0.111] d2[0.222] g[36.840]
>16964, d1[0.112] d2[0.552] g[86.997]
>16965, d1[0.517] d2[0.196] g[60.260]
>16966, d1[0.110] d2[0.187] g[42.237]
>16967, d1[0.101] d2[0.256] g[117.827]
>16968, d1[0.109] d2[0.165] g[146.320]
>16969, d1[0.110] d2[0.216] g[96.675]
>16970, d1[0.148] d2[0.215] g[234.847]
>16971, d1[0.550] d2[0.224] g[21.194]
>16972, d1[0.341] d2[0.353] g[2.339]
>16973, d1[0.124] d2[0.260] g[116.733]
>16974, d1[0.147] d2[0.146] g[64.009]
>16975, d1[0.154] d2[0.230] g[111.019]
>16976, d1[0.109] d2[0.542] g[188.694]
>16977, d1[0.100] d2[0.689] g[48.259]
>16978, d1[0.199] d2[0.196] g[34.642]
>16979, d1[0.629] d2[0.248] g[68.552]
>16980, d1[0.152] d2[0.339] g[143.892]
>16981, d1[0.176] d2[0.209] g[143.344]
>16982, d1[0.644] d2[0.211] g[115.954]
>16983, d1[0.625] d2[0.161] g[53.475]
>1

>17172, d1[0.188] d2[0.286] g[33.807]
>17173, d1[0.502] d2[0.198] g[45.846]
>17174, d1[0.163] d2[0.361] g[111.401]
>17175, d1[0.159] d2[0.287] g[121.750]
>17176, d1[0.161] d2[0.253] g[24.012]
>17177, d1[0.196] d2[0.175] g[183.131]
>17178, d1[0.175] d2[0.228] g[130.830]
>17179, d1[0.422] d2[0.218] g[16.414]
>17180, d1[0.472] d2[0.268] g[39.112]
>17181, d1[0.279] d2[0.217] g[42.491]
>17182, d1[0.233] d2[0.215] g[39.800]
>17183, d1[0.169] d2[0.284] g[29.451]
>17184, d1[0.341] d2[0.260] g[69.641]
>17185, d1[0.275] d2[0.316] g[183.712]
>17186, d1[0.204] d2[0.602] g[56.948]
>17187, d1[0.197] d2[0.242] g[40.512]
>17188, d1[0.232] d2[0.200] g[151.007]
>17189, d1[0.567] d2[0.332] g[69.602]
>17190, d1[0.190] d2[0.571] g[29.082]
>17191, d1[0.246] d2[0.274] g[180.811]
>17192, d1[0.233] d2[0.231] g[153.841]
>17193, d1[0.214] d2[0.361] g[77.103]
>17194, d1[0.180] d2[0.527] g[188.428]
>17195, d1[0.501] d2[0.340] g[32.694]
>17196, d1[0.192] d2[0.233] g[249.796]
>17197, d1[0.228] d2[0.218] g[42.742]
>1

>17387, d1[0.106] d2[0.655] g[153.220]
>17388, d1[0.103] d2[0.179] g[198.216]
>17389, d1[0.197] d2[0.242] g[107.341]
>17390, d1[0.472] d2[0.223] g[46.081]
>17391, d1[0.133] d2[0.217] g[89.650]
>17392, d1[0.137] d2[0.210] g[104.674]
>17393, d1[0.184] d2[0.202] g[55.242]
>17394, d1[0.256] d2[0.204] g[61.896]
>17395, d1[0.140] d2[0.221] g[45.518]
>17396, d1[0.139] d2[0.153] g[54.901]
>17397, d1[0.131] d2[0.164] g[134.555]
>17398, d1[0.171] d2[0.174] g[28.361]
>17399, d1[0.171] d2[0.149] g[211.049]
>17400, d1[0.127] d2[0.146] g[56.563]
>17401, d1[0.597] d2[0.209] g[21.083]
>17402, d1[0.137] d2[0.161] g[55.409]
>17403, d1[0.090] d2[0.141] g[115.121]
>17404, d1[0.070] d2[0.212] g[68.442]
>17405, d1[0.135] d2[0.201] g[101.252]
>17406, d1[0.092] d2[0.275] g[41.835]
>17407, d1[0.069] d2[0.197] g[68.077]
>17408, d1[0.140] d2[0.312] g[57.670]
>17409, d1[0.137] d2[0.243] g[172.592]
>17410, d1[0.457] d2[0.164] g[32.815]
>17411, d1[0.095] d2[0.160] g[93.585]
>17412, d1[0.091] d2[0.171] g[46.850]
>17

>17601, d1[0.131] d2[0.255] g[47.464]
>17602, d1[0.511] d2[0.145] g[48.499]
>17603, d1[0.348] d2[0.399] g[2.013]
>17604, d1[0.422] d2[0.339] g[145.602]
>17605, d1[0.290] d2[0.207] g[84.048]
>17606, d1[0.149] d2[0.157] g[27.671]
>17607, d1[0.584] d2[0.215] g[31.688]
>17608, d1[0.445] d2[0.215] g[56.534]
>17609, d1[0.564] d2[0.203] g[38.137]
>17610, d1[0.194] d2[0.186] g[68.718]
>17611, d1[0.193] d2[0.157] g[96.387]
>17612, d1[0.409] d2[0.202] g[283.325]
>17613, d1[0.588] d2[0.243] g[39.624]
>17614, d1[0.203] d2[0.482] g[50.800]
>17615, d1[0.254] d2[0.204] g[182.278]
>17616, d1[0.109] d2[0.192] g[96.895]
>17617, d1[0.397] d2[0.329] g[8.729]
>17618, d1[0.144] d2[0.224] g[52.582]
>17619, d1[0.198] d2[0.234] g[138.656]
>17620, d1[0.142] d2[0.232] g[75.038]
>17621, d1[0.184] d2[0.221] g[59.190]
>17622, d1[0.536] d2[0.204] g[56.838]
>17623, d1[0.186] d2[0.208] g[83.337]
>17624, d1[0.248] d2[0.188] g[37.734]
>17625, d1[0.080] d2[0.172] g[115.948]
>17626, d1[0.137] d2[0.211] g[18.798]
>17627, d

KeyboardInterrupt: 

In [19]:
#end the tensorflow session
from tensorflow.keras import backend as K
K.clear_session()

In [20]:
#for cuda to release GPU memory
from numba import cuda
cuda.select_device(0)
cuda.close()